# Создание рекомендательной системы для авторов музыки

## Описание задания
В роли ML-инженера в сервисе музыкального стриминга вам представлена задача создания рекомендательной системы для собственного потокового сервиса, аналогичного Spotify. В этот раз вы не обладаете знаниями о контенте, зато у вас есть история прослушиваний исполнителей для каждого пользователя.

Ваша задача состоит в улучшении алгоритма, который определит наиболее релевантные рекомендации для каждого пользователя на основе их истории прослушиваний.

**Описание данных**  
В файле `train.parquet` вам предоставлены данные о прослушиваниях пользователями артистов в сервисе.

Зачастую в реальных системах данные анонимизированы, поэтому и в этом задании мы будем работать только с абстрактными идентификаторами. Поэтому для оценки метода мы будем использовать рассмотренные ранее метрики качества.

|Поле|Тип|Описание|
|---|---|---|
|user_id|str|Идентификатор пользователя|
|artist_id|str|Идентификатор исполнителя|

## Оценивание
Оценка за задание зависит от метрики качества ndcg@20, которая часто применяется в задаче ранжирования. Чем больше релевантных объектов находится в начале списка рекомендаций, тем она выше. Для оценивания используются сессионная валидация с последними N артистами из истории прослушиваний.

В зависимости от значения этой метрики, финальная оценка будет линейно отображена из диапазона значений метрики [0.02, 0.07] в диапазон оценки [0, 25]. То есть при ndcg@20 <= 0.02 вы получите оценку 0, при ndcg@20 >= 0.07 вы получите оценку 25, а все промежуточные значения считаются по формуле




$  
<font size="5">
$25\frac{ndcg−0.02}{0.07−0.02}$
</font>


## Советы по решению
- параметры моделей по умолчанию зачастую не являются оптимальными, стоит подобрать наиболее подходящие
- для улучшения метрик стоит обучать модель на всех возможных данных (train + test)
- в качестве комбинации методов можно использовать взвешенную сортировку, где вес зависит от позиции в списке рекомендаций и «качества» метода

При офлайн оценке рекомендуется оценивать ваше решение с помощью валидации по событиям. Не забывайте, что при сдаче решения стоит обучить вашу модель на всех возможных данных для увеличения метрики.

In [23]:
import pandas as pd
import polars as pl
import numpy as np
import scipy.sparse as sp
from gensim.models import Word2Vec
import optuna
import random

from tqdm import tqdm
from typing import List, Any

/home/jupyter-pavel.lyubovin/.conda/envs/recsys/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Читаем датасет

In [3]:
data = pl.read_parquet('train.parquet')
data

user_id,artist_id
str,str
"""d705b538-1bd8-…","""69c71d72-7ed8-…"
"""d705b538-1bd8-…","""30bf469f-9abd-…"
"""d705b538-1bd8-…","""a26c9335-2459-…"
"""d705b538-1bd8-…","""69c903b5-dff0-…"
"""d705b538-1bd8-…","""af8eef9d-13aa-…"
"""d705b538-1bd8-…","""293a86ee-6ce7-…"
"""d705b538-1bd8-…","""348f4909-1c48-…"
"""d705b538-1bd8-…","""ad2bf122-726e-…"
"""d705b538-1bd8-…","""cc97fc57-30b5-…"


## Метрики

В этом задании нашей задачей будем оптимизация метрик ndcg@20. Тем не менее, такая метрику сложно интерпретировать и поэтому вам также будет доступно значение метрики hitrate@20, с которой вы уже познакомились в прошлой домашке

In [4]:
TOP_K = 20


def user_hitrate(y_relevant: List[str], y_recs: List[str], k: int = TOP_K) -> int:
    return int(len(set(y_relevant).intersection(y_recs[:k])) > 0)

def user_ndcg(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: ndcg metric for user recommendations
    """
    dcg = sum([1. / np.log2(idx + 2) for idx, item in enumerate(y_rec[:k]) if item in y_rel])
    idcg = sum([1. / np.log2(idx + 2) for idx, _ in enumerate(zip(y_rel, np.arange(k)))])
    return dcg / idcg

В этом датасете идентификаторы представлены в виде строк, но для работы с ними может быть проще сделать преобразование в числа (например, для алгоритмов матричной факторизации)

In [5]:
user_mapping = {k: v for v, k in enumerate(data['user_id'].unique())}
user_mapping_inverse = {k: v for v, k in user_mapping.items()}

artist_mapping = {k: v for v, k in enumerate(data['artist_id'].unique())}
artist_mapping_inverse = {k: v for v, k in artist_mapping.items()}

In [6]:
grouped_df_with_inds = (
    data
    .with_columns([
        pl.col('user_id').apply(user_mapping.get),
        pl.col('artist_id').apply(artist_mapping.get),
    ])
    # для каждого пользователя оставим последние 3 объекта в качестве тестовой выборки,
    # а остальное будем использовать для тренировки
    .groupby('user_id')
    .agg([
        pl.col('artist_id').apply(lambda x: x[:-3]).alias('train_item_ids'),
        pl.col('artist_id').apply(lambda x: x[-3:]).alias('test_item_ids'),
    ])
)

grouped_df_with_inds

user_id,train_item_ids,test_item_ids
i64,list[i64],list[i64]
47904,"[10482, 34566, … 30556]","[54222, 42063, 74243]"
40768,"[82671, 15478, … 59985]","[21027, 53132, 32472]"
6688,"[42853, 2989, … 63041]","[62040, 73224, 53287]"
24192,"[78377, 43526, … 10850]","[13721, 17598, 10740]"
26496,"[88898, 43382, … 18668]","[19284, 24769, 68274]"
41696,"[47343, 28827, … 1101]","[7217, 24463, 46178]"
23712,"[26317, 55268, … 57413]","[722, 37616, 38173]"
32096,"[43837, 37616, … 41796]","[45284, 1059, 36408]"
6432,"[13569, 84848, … 60721]","[85959, 19247, 31304]"


In [7]:
median_seq_len = int(grouped_df_with_inds['train_item_ids'].apply(len).median())
print(f"средняя длина сессии {median_seq_len}")

средняя длина сессии 42


In [8]:
# соберем строчки для разреженной матрицы
rows = []
cols = []
values = []
for user_id, train_ids, _ in grouped_df_with_inds.rows():
    rows.extend([user_id] * len(train_ids))
    values.extend([1] * len(train_ids))
    cols.extend(train_ids)

user_item_data = sp.csr_matrix((values, (rows, cols)))

## Бейзлайны

В качестве простого бейзлайна будем рекомендовать самый популярных артистов

Мы хотим сначала провалидировать такое решение, а значит в качестве популярных артистов мы возьмем только тех, кто чаще встречается в `train_item_ids`

In [9]:
top_artists = (
    grouped_df_with_inds
    .select(pl.col('train_item_ids').alias('artist_id'))
    .explode('artist_id')
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(TOP_K + median_seq_len)
)['artist_id'].to_list()

In [10]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = top_artists.copy()
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

NDCG@20 = 0.01401, Hitrate@20 = 0.10248


Не забываем про фильтрацию просмотренного (для разных доменов и подходов это не всегда улучши рекомендации, но в данном случае дало прирост)

In [11]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = [artist_id for artist_id in top_artists if artist_id not in user_history]
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

NDCG@20 = 0.01740, Hitrate@20 = 0.11684


## Построим файл с рекомендациями

Для построения рекомендаций теперь можем учесть все возможные данные

In [12]:
top_artists = (
    data
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(TOP_K + median_seq_len)
)['artist_id'].to_list()

In [13]:
submission = []

for user_id, user_history in data.groupby('user_id').agg(pl.col('artist_id')).rows():
    y_rec = top_artists.copy()
    
    submission.append((user_id, y_rec))
    
submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('sample_submission.parquet')
submission

user_id,y_rec
str,list[str]
"""1daa0aae-746f-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""74203af4-7674-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""2b4d6ce1-99e3-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""6971db2c-dadf-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""f7faf00d-aa96-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""f6987c10-10a0-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""38cc4f02-0efb-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""b7939dca-9492-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""d57f33f1-e594-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"


Не забывайте, что в файле с рекомендациями должны быть **исходные идентификаторы (строки)**, а не преобразованные в числа!

# Начинаем выполнять задание

In [20]:
grouped_df_with_inds

user_id,train_item_ids,test_item_ids
i64,list[i64],list[i64]
47904,"[10482, 34566, … 30556]","[54222, 42063, 74243]"
40768,"[82671, 15478, … 59985]","[21027, 53132, 32472]"
6688,"[42853, 2989, … 63041]","[62040, 73224, 53287]"
24192,"[78377, 43526, … 10850]","[13721, 17598, 10740]"
26496,"[88898, 43382, … 18668]","[19284, 24769, 68274]"
41696,"[47343, 28827, … 1101]","[7217, 24463, 46178]"
23712,"[26317, 55268, … 57413]","[722, 37616, 38173]"
32096,"[43837, 37616, … 41796]","[45284, 1059, 36408]"
6432,"[13569, 84848, … 60721]","[85959, 19247, 31304]"


In [22]:
def evaluate_model(model):
    ndcg_list = []
    hitrate_list = []
    for train_ids, y_rel in grouped_df_with_inds.select('train_item_ids', 'test_item_ids').rows():
        model_preds = model.predict_output_word(
            train_ids, topn=(TOP_K + len(train_ids))
        )
        if model_preds is None:
            hitrate_list.append(0)
            continue

        y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
        ndcg_list.append(user_ndcg(y_rel, y_rec))
        hitrate_list.append(user_hitrate(y_rel, y_rec))
    return np.mean(ndcg_list), np.mean(hitrate_list)

# обучим w2v с параметрами по умолчанию
model = Word2Vec(grouped_df_with_inds['train_item_ids'].to_list())
mean_ndcg, mean_hitrate = evaluate_model(model)
print(f'Ndcg@{TOP_K} = {mean_ndcg:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')

Ndcg@20 = 0.0237 Hitrate@20 = 0.1490


Пробую подобрать параметры оптуной

In [24]:
SEED = 42

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

def objective(trial):
    sg = trial.suggest_categorical('sg', [0, 1])
    window = trial.suggest_int('window', 1, 10)
    ns_exponent = trial.suggest_float('ns_exponent', -3, 3)
    negative = trial.suggest_int('negative', 3, 20)
    min_count = trial.suggest_int('min_count', 0, 20)
    vector_size = trial.suggest_categorical('vector_size', [16, 32, 64, 128])
    
    print({
        'sg': sg,
        'window_len': window,
        'ns_exponent': ns_exponent,
        'negative': negative,
        'min_count': min_count,
        'vector_size': vector_size,
    })
    
    set_seed(SEED)
    model = Word2Vec(
        grouped_df_with_inds['train_item_ids'].to_list(),
        window=window,
        sg=sg,
        hs=0,
        min_count=min_count,
        vector_size=vector_size,
        negative=negative,
        ns_exponent=ns_exponent,
        seed=SEED,
        epochs=10,
    )
    
    mean_ndcg, mean_hitrate = evaluate_model(model)
    print(f'NDCG@{TOP_K} = {mean_ndcg:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')
    return mean_ndcg
    
    
study = optuna.create_study(directions=('maximize',))
study.optimize(objective, n_trials=100)

study.best_params

[I 2024-04-04 18:14:42,307] A new study created in memory with name: no-name-5ea8202a-6b22-4868-81c2-06edfd5feda6


{'sg': 1, 'window_len': 10, 'ns_exponent': -0.043770927479415356, 'negative': 10, 'min_count': 5, 'vector_size': 64}


[I 2024-04-04 18:18:43,309] Trial 0 finished with value: 0.04498511241730642 and parameters: {'sg': 1, 'window': 10, 'ns_exponent': -0.043770927479415356, 'negative': 10, 'min_count': 5, 'vector_size': 64}. Best is trial 0 with value: 0.04498511241730642.


NDCG@20 = 0.0450 Hitrate@20 = 0.2694
{'sg': 0, 'window_len': 8, 'ns_exponent': -0.5597146402069706, 'negative': 17, 'min_count': 10, 'vector_size': 16}


[I 2024-04-04 18:20:09,137] Trial 1 finished with value: 0.04137813591136338 and parameters: {'sg': 0, 'window': 8, 'ns_exponent': -0.5597146402069706, 'negative': 17, 'min_count': 10, 'vector_size': 16}. Best is trial 0 with value: 0.04498511241730642.


...

[I 2024-04-04 20:35:19,867] Trial 98 finished with value: 0.028900413071824664 and parameters: {'sg': 0, 'window': 8, 'ns_exponent': 0.7580884685506912, 'negative': 4, 'min_count': 18, 'vector_size': 32}. Best is trial 46 with value: 0.060584399847025426.


NDCG@20 = 0.0289 Hitrate@20 = 0.1734
{'sg': 0, 'window_len': 9, 'ns_exponent': 0.224215400672453, 'negative': 3, 'min_count': 19, 'vector_size': 32}


[I 2024-04-04 20:36:05,605] Trial 99 finished with value: 0.05183632194671007 and parameters: {'sg': 0, 'window': 9, 'ns_exponent': 0.224215400672453, 'negative': 3, 'min_count': 19, 'vector_size': 32}. Best is trial 46 with value: 0.060584399847025426.


NDCG@20 = 0.0518 Hitrate@20 = 0.3266


{'sg': 0,
 'window': 9,
 'ns_exponent': 0.0870309781408407,
 'negative': 7,
 'min_count': 14,
 'vector_size': 64}

In [25]:
study.best_params

{'sg': 0,
 'window': 9,
 'ns_exponent': 0.0870309781408407,
 'negative': 7,
 'min_count': 14,
 'vector_size': 64}

In [26]:
# Возьму лучшие параметры и обучу побольше эпох
set_seed(SEED)
model = Word2Vec(
    grouped_df_with_inds['train_item_ids'].to_list(),
    window=9,
    sg=0,
    hs=0,
    min_count=14,
    vector_size=64,
    negative=7,
    ns_exponent=0.0870309781408407,
    seed=SEED,
    epochs=50,
)
mean_ndcg, mean_hitrate = evaluate_model(model)
print(f'NDCG@{TOP_K} = {mean_ndcg:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')

NDCG@20 = 0.0658 Hitrate@20 = 0.3778


In [27]:
# Может 100 эпох лучше, чем 50?
set_seed(SEED)
model = Word2Vec(
    grouped_df_with_inds['train_item_ids'].to_list(),
    window=9,
    sg=0,
    hs=0,
    min_count=14,
    vector_size=64,
    negative=7,
    ns_exponent=0.0870309781408407,
    seed=SEED,
    epochs=100,
)
mean_ndcg, mean_hitrate = evaluate_model(model)
print(f'NDCG@{TOP_K} = {mean_ndcg:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')

NDCG@20 = 0.0668 Hitrate@20 = 0.3804


Лучше, но совсем чуть-чуть

In [29]:
# Обучим модель на полном датасете
grouped_df_with_inds_full = (
    data
    .with_columns([
        pl.col('user_id').apply(user_mapping.get),
        pl.col('artist_id').apply(artist_mapping.get),
    ])
    # для каждого пользователя оставим последние 3 объекта в качестве тестовой выборки,
    # а остальное будем использовать для тренировки
    .groupby('user_id')
    .agg([
        pl.col('artist_id').apply(lambda x: x[:]).alias('item_ids_full'),
        # pl.col('artist_id').apply(lambda x: x[-3:]).alias('test_item_ids'),
    ])
)

grouped_df_with_inds_full

user_id,item_ids_full
i64,list[i64]
47136,"[19175, 34112, … 71009]"
18240,"[33648, 61027, … 85541]"
39104,"[12004, 36675, … 79429]"
41728,"[71848, 48486, … 48572]"
16416,"[23689, 58652, … 37254]"
46560,"[1206, 69633, … 75239]"
48000,"[19427, 31911, … 75381]"
49408,"[42731, 17598, … 68410]"
14336,"[63397, 51530, … 49337]"


In [30]:
set_seed(SEED)
model_full = Word2Vec(
    grouped_df_with_inds_full['item_ids_full'].to_list(),
    window=9,
    sg=0,
    hs=0,
    min_count=14,
    vector_size=64,
    negative=7,
    ns_exponent=0.0870309781408407,
    seed=SEED,
    epochs=100,
)

In [41]:
# Делаю предсказания и запись в файл
submission = []

for user_id, item_ids_full in grouped_df_with_inds_full.select('user_id', 'item_ids_full').rows():
    model_preds = model_full.predict_output_word(item_ids_full, topn=(TOP_K + len(item_ids_full)))
    y_rec = [artist_mapping_inverse[pred[0]] for pred in model_preds if pred[0] not in item_ids_full]
    submission.append((user_mapping_inverse[user_id], y_rec))

submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('submission.parquet')
submission

user_id,y_rec
str,list[str]
"""bfba5702-a137-…","[""0e78b958-b53d-490f-bc6d-5ced445ad2c0"", ""519544ab-b48a-4c29-a933-c39af3dc7d97"", … ""7e0a77d9-a8f8-4a60-ac5d-f771056f8306""]"
"""31969f3d-29ea-…","[""c92b1ba4-c710-4c2d-82f8-d446f409548a"", ""a6cdae3d-16ae-430a-8716-b82f64ed758d"", … ""35f1b9df-4c83-4281-b27c-48b7a214de7d""]"
"""e19c4591-07f0-…","[""4ecc576e-e92a-47b2-983e-ecacdb210ae7"", ""0775b8b6-2b02-41f7-a2ce-f0a5c0a64650"", … ""ea908d75-6b47-4442-957f-eaac87538adf""]"
"""308b68c5-60b5-…","[""5013361c-4240-442e-855e-19f0185aa442"", ""7c5b4e7e-4929-4d34-a55b-e28e28a8ea3c"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""c283798b-f59d-…","[""d5bcd7a1-b02c-43ce-90b3-9c794cf4d596"", ""757a27fe-89a2-47a5-a2a8-a4dd864737b3"", … ""67bbde6c-2555-4b5c-a2d9-7d5d54b26975""]"
"""070047bd-5ad1-…","[""90ddb877-05d9-4b70-884b-8d90082d2868"", ""0f692d11-9a2a-4964-bfaa-3841b4d537cb"", … ""83ed959d-15bb-4173-a347-75f4d498466c""]"
"""ece51a1d-9f3b-…","[""6f1b8a85-c127-40a6-84be-d659aefc99e8"", ""c8822a9f-f6f2-4025-ae84-8b7cd45c916f"", … ""ee0f3f04-8fa0-46ad-b821-2a5ebf5ca6e9""]"
"""5d977cb0-b3d6-…","[""ee0f3f04-8fa0-46ad-b821-2a5ebf5ca6e9"", ""d1947eb3-8fba-4084-a173-a514820ee4cc"", … ""76f67ddb-f1d0-4502-abf7-42224deb5cda""]"
"""8d65c9ed-c6d1-…","[""a8e78fd5-610f-4de6-8a5f-3e098f6cb151"", ""b7999c99-13c5-4b43-a9e9-d60d7615399c"", … ""ed471306-1aea-4695-be38-d5833ed5db12""]"


## Финальный результат

Баллы:  25.00  

Total score:  
Passed with 100%!  

Tests result:  
Test	Time	Score  
✓ test_csv_file	2.2s	100 / 100  
Messages:  
Test	Message  
✓ test_csv_file	NDCG@20 = 0.08343251208200754  
Hitrate@20 = 0.3697  